# DSNet:  Automatic Dermoscopic Skin Lesion Segmentation


* Through  this  study,  we  present  a  new  and  automatic  semantic  segmentation  network for robust skin lesion segmentation named Dermoscopic Skin Network (DSNet).

* In order to reduce the number of parameters to make the network lightweight, we used a depth-wise separable convolution in lieu of standard convolution to project the learnt discriminating features onto the pixel space at different stages of the encoder.

* For any query: 
        ** Md. Kamrul Hasan 
        ** M.Sc. in Medical Imaging and Applications (MAIA)
        ** Erasmus Scholar [2017-2019] 
        ** Contact: kamruleeekuet@gmail.com

## Import all the python Packages 

In [0]:
#------------------------------Keras Packages----------------------------------- 
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.losses import *
from keras import backend as keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.densenet import DenseNet201, DenseNet121
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator


#------------------------------Others python Packages---------------------------
import numpy as np # High-level mathematical functions for n-dimensional arrays 
import os   
import cv2
import skimage.io as io
import skimage.transform as trans
import numpy as np
import glob
from PIL import Image
import skimage
from keras.initializers import Constant
from matplotlib import pyplot as plt
%matplotlib inline
from skimage.morphology import disk
from sklearn.metrics import confusion_matrix
from skimage.measure import label, regionprops
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import jaccard_similarity_score

C:\Users\kamru\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Designing of Proposed DSNet

In [0]:
def DSNet(nClasses, input_height, input_width):

    #------------------------------Define Input Shape----------------------------------
    
    img_input = Input(shape=(input_height, input_width, 3)) # defining the Input shape 
    
    #Load DenseNet121 from keras. This model is initialized with the ImageNet. 
    #This part is responsible for the feature extraction which is so called convolution
    # part of the semantic segmentation (Encoder). 
    
    Encoder_Dense = DenseNet121( weights = 'imagenet',
						include_top = False,
						input_tensor = img_input) 

    Encoder = SeparableConv2D(filters = 1024,
					kernel_size = (3, 3),
					activation = 'relu',
					kernel_initializer='he_normal',
					padding="same")(Encoder_Dense.output)
    Encoder = BatchNormalization()(Encoder)

    
    # Decoding the encoded features to reconstruct the original input shape Image.

    Decoder = UpSampling2D(size = (2, 2))(Encoder)
    Decoder = concatenate([Encoder_Dense.get_layer(name="pool3_pool").output, Decoder], axis=-1)
    Decoder = SeparableConv2D(filters = 1024,
						kernel_size = (3, 3),
						activation = 'relu',
						kernel_initializer='he_normal',
						padding = "same")(Decoder)
    Decoder = BatchNormalization()(Decoder)
    
    Decoder = UpSampling2D(size = (2, 2))(Decoder)
    Decoder = concatenate([Encoder_Dense.get_layer(name="pool2_pool").output, Decoder], axis=-1)
    Decoder = SeparableConv2D(filters = 512,
						kernel_size = (3, 3),
						activation = 'relu',
						kernel_initializer='he_normal',
						padding = "same")(Decoder)
    Decoder = BatchNormalization()(Decoder)

    Decoder = UpSampling2D(size = (2, 2))(Decoder)
    Decoder = concatenate([Encoder_Dense.get_layer(name="pool1").output, Decoder], axis=-1)
    Decoder = SeparableConv2D(filters = 256,
						kernel_size = (3, 3),
						activation = 'relu',
						kernel_initializer='he_normal',
						padding = "same")(Decoder)
    Decoder = BatchNormalization()(Decoder)

    Decoder = UpSampling2D( size = (2, 2))(Decoder)
    Decoder = concatenate([Encoder_Dense.get_layer(name="conv1/bn").output, Decoder], axis=-1)
    Decoder = SeparableConv2D(filters = 128,
						kernel_size = (3, 3),
						activation = 'relu',
						kernel_initializer='he_normal',
						padding = "same")(Decoder)
    Decoder = BatchNormalization()(Decoder)

    Decoder = UpSampling2D(size = (2, 2))(Decoder)    
    Decoder = SeparableConv2D(filters = 64,
						kernel_size = (3, 3),
						activation = 'relu',
						kernel_initializer='he_normal',
						padding = "same")(Decoder)
    Decoder = BatchNormalization()(Decoder)

    Decoder = SeparableConv2D(filters = nClasses,
						kernel_size = (1, 1),
						activation = 'relu',
						kernel_initializer='he_normal',
						padding = "same")(Decoder)
    Decoder = BatchNormalization()(Decoder)
    

    Predicted_Mask = Conv2D(filters = 1,
									kernel_size = 1,
									activation = 'sigmoid')(Decoder)
    
    
    DSNet_model = Model(input = img_input, output = Predicted_Mask)

    return DSNet_model

## Data Pre-processing 

In [0]:
def DataPreProcessing(img, mask):
    img = preprocess_input(img) # Data standardization meaning 0 mean and unit variance.
    img = img/img.max()         # Data Normalization
    mask = mask/mask.max()      # Mask Normalization
    return (img, mask)          # Return Tuple of Original image along with GT image. 

## Function for Train generator 

In [0]:
def trainGenerator(batch_size,
                   train_path,
                   image_folder,
                   mask_folder,
                   aug_dict,
                   image_color_mode = "rgb",
                   mask_color_mode = "grayscale",
                   image_save_prefix  = "image",
                   mask_save_prefix  = "mask",
                   save_to_dir = None,
                   target_size = (192,256),
                   seed = 1):
    
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = 'rgb',
        target_size = target_size,
        batch_size = batch_size,
#         save_to_dir = save_to_dir,
#         save_prefix  = image_save_prefix,
        seed = seed)

    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = target_size,
        batch_size = batch_size,
#         save_to_dir = save_to_dir,
#         save_prefix  = mask_save_prefix,
        seed = seed)

    
    train_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in train_generator:
        img,mask = DataPreProcessing(img,mask)
        yield (img,mask)

## Function for Test and Validation generator 

In [0]:
def ValGenerator(batch_size,
                 val_path,
                 image_folder,
                 mask_folder,
                 target_size = (192,256),
                 seed = 1):
    
    image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator()
    
    image_generator = image_datagen.flow_from_directory(
        val_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = 'rgb',
        target_size = target_size,
        batch_size = batch_size,
        seed = seed)

    
    mask_generator = mask_datagen.flow_from_directory(
        val_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = target_size,
        batch_size = batch_size,
        seed = seed)

    
    val_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in val_generator:
        img,mask = DataPreProcessing(img,mask)
        yield (img,mask)

## Function for Loss

In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def Jaccard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection ) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection)

def Jaccard_coef_loss(y_true, y_pred):
    return (1-Jaccard_coef(y_true, y_pred))

def bcc_Jaccard_coef_loss(y_true, y_pred):
    return (binary_crossentropy(y_true, y_pred)+(1-Jaccard_coef(y_true, y_pred)))

## Train and Validate the Model

In [0]:
# This is for the list of the augmentation (Geometric Augmentation). This 
# geometric augmentation will increase the numbers of training images to
# overcome over-fiting and curse of dimensionality. This will act as a online 
# aumentation that will help to reduce the memory consumtion during the 
# Trainig phase. If you want you can augment the images before and save it to
# the directory and read them.

data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

CurrentDirectory=os.getcwd()

height=192
width=256

# Calling the train generator function with required arguments. 
TrainGen= trainGenerator(batch_size=5,
                         train_path=CurrentDirectory+'/',
                         image_folder='Train_IMAGE_ISIC_2017',
                         mask_folder='Train_GT_ISIC_2017',
                         aug_dict=data_gen_args,
                         image_color_mode = "grayscale",
                         mask_color_mode = "grayscale",
                         image_save_prefix  = "image",
                         mask_save_prefix  = "mask",
#                          save_to_dir = CurrentDirectory+'/aug/',
                         target_size = (height,width),
                         seed = 1)
 
# Calling the Validation/ Test generator function with required arguments. 
TestGen= ValGenerator(batch_size=5,
                      val_path=CurrentDirectory+'/',
                      image_folder='Val_IMAGE_ISIC_2017', 
                      mask_folder='Val_GT_ISIC_2017',
                      target_size = (height,width),
                      seed = 1)


# Calling the designed Model to train.

model = DSNet(2, height, width)

model.compile(optimizer = 'adadelta',
                loss = bcc_Jaccard_coef_loss,
                metrics = [Jaccard_coef])

# This will plot a graph of the model and save it to a file. 
plot_model(model, show_shapes=True, to_file='model.png')

# This will print the model summary and total parameters for each operations
# along with total learnable parameters.
model.summary()


# Save the model after every epoch. You can save only the best model. If the 
# specified metric do not improve it will not save the that current epoch. 

model_checkpoint = ModelCheckpoint('Trained_Model.hdf5',
                                   monitor='val_Jaccard_coef',
                                   verbose=1,
                                   mode='max',
                                   save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_Jaccard_coef',
                              factor=0.5,
                              patience=8,
                              verbose=1,
                              mode='max',
                              min_lr=0.00000001)

early_stopping = EarlyStopping(monitor='val_Jaccard_coef',
                               patience=5,
                               verbose=1,
                               mode='max',
                               restore_best_weights=True)


# Trains the model on data generated batch-by-batch by a Python generator 
# (or an instance of Sequence). The generator is run in parallel to the model,
# for efficiency. For instance, this allows you to do real-time data
# augmentation on images on CPU in parallel to training your model on GPU.


# history=model.fit_generator(TrainGen, 
#                             steps_per_epoch=400, 
#                             epochs=200, 
#                             verbose=1, 
#                             validation_data= TestGen, 
#                             validation_steps=30, 
#                             callbacks=[model_checkpoint, reduce_lr])


C:\Users\kamru\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 192, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 198, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 96, 128, 64)  9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 96, 128, 64)  256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

## Plotting the train and validation performance

In [0]:
# # Plot training & validation accuracy values
# plt.plot(history.history['Jaccard_coef'])
# plt.plot(history.history['val_Jaccard_coef'])
# plt.title('Model accuracy')
# plt.ylabel('dice_coef')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper right')
# plt.grid('on')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper right')
# plt.grid('on')
# plt.show()

In [0]:
def Performance_Metrics(y_true, y_pred):
    ''' 
    The Dice similarity coefficient (DSC) is a statistical validation metric for the image
    segmentations and determines the spatial overlap accuracy of the segmentation. It also 
    same as F1-score. 
    
    The Intersection over Union (IoU) also referred to as the Jaccard index (JI), is essentially
    a method to quantify the percent overlap between the GT mask and prediction output.
    This metric is closely related to the DSC. The IoU metric measures the number of pixels
    common between the target and prediction masks divided by the total number of pixels present
    across both masks.
    
    Mathematically, they are related as follows:
    IoU = DSC/(2-DSC) or DSC = 2*IoU/(IoU+1)
    
    The Sensitivity also called the true positive rate, the recall, or probability of detection that 
    measures the proportion of actual positives that are correctly identified as such class 
    (e.g., the percentage of sick people who are correctly identified as having the condition).
    
    The Specificity also called the true negative rate measures the proportion of actual negatives that
    are correctly identified as such class. (e.g., the percentage of healthy people who are correctly
    identified as not having the condition).
    
  
    Input Arguments: 
        y_true: True Labels of the 2D images so called ground truth (GT).
        y_pred: Predicted Labels of the 2D images so called Predicted/ segmented Mask.
        
    Output Arguments: 
        dsc: The DSC between y_true and y_pred
        iou: The IoU between y_true and y_pred
        accuracy: The accuracy of pixels clasification.
        sensitivity: The accuracy of foreground pixels clasification.
        specificity: The accuracy of background pixels clasification.
        balancedAccuracy: The balanced accuracy of both foreground & background pixels clasification.
        
    Author: Md. Kamrul Hasan, 
            Erasmus Scholar on Medical Imaging and Application (MAIA)
            E-mail: kamruleeekuet@gmail.com

    '''
    if y_true.shape != y_pred.shape:
        raise ValueError("Shape mismatch!! y_true and y_pred must have the same shape.")

    y_true_f = (y_true/y_true.max()).flatten()
    y_pred_f = (y_pred/y_pred.max()).flatten()
    
    intersection = np.sum(y_true_f * y_pred_f)
    
    dsc = (2. * intersection ) / (np.sum(y_true_f) + np.sum(y_pred_f))
    iou = (intersection) / (np.sum(y_true_f) + np.sum(y_pred_f)-intersection)
    
    y_true = np.asarray(y_true).astype(np.bool)
    y_pred = np.asarray(y_pred).astype(np.bool)

        
    y_true=y_true.flatten()
    y_pred=y_pred.flatten()
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (tn+tp)/(tn+fp+fn+tp)
    sensitivity = (tp)/(fn+tp)
    specificity = (tn)/(tn+fp)
    balancedAccuracy = (sensitivity+specificity)/2

    
    return dsc, iou, accuracy, sensitivity, specificity, balancedAccuracy

## Mask Prediction from DSNet

In [0]:
# Predict the masks using the proposed DSNet and those masks are processed using 
# the “DSNet_Performance.m” to estimate the performance metrics. The ROC curves 
# are plotted using the “ROC_DSNet.ipynb” 

testImagePath =CurrentDirectory+"/Test_IMAGE_ISIC_2017/"
imagePath = glob.glob(testImagePath+"*.jpg")
imagePath.sort()
print(len(imagePath))

GTPath= CurrentDirectory+"/Test_GT_ISIC_2017/"
GT = glob.glob(GTPath+"*.png")
GT.sort()
print(len(GT))


MaskSavePath= CurrentDirectory+'/Mask/'

model = UNet(2, height, width)

model.load_weights("Trained_Model.hdf5")

for imageName,gt in zip(imagePath,GT):
    img = cv2.imread(imageName,-1)
    imgcopy=img.copy()
    filename, file_extension = os.path.splitext(imageName) 
    
    img=cv2.resize(img,(width,height))
    img=img.astype(np.float32)
    img = img/img.max()
    img=np.expand_dims(img, axis=0)
    Prediction=model.predict(img,verbose=0)
    
    mask=Prediction.reshape(height,width)
    mask=(255*(mask/mask.max())).astype('uint8')
    cv2.imwrite(MaskSavePath+filename[-12:]+'.png',mask)

200
200
